In [27]:
import sys
sys.path.append('..')
from utils import split_data
import torch
from model import cifar_mobilenet, vgg16
import torch.nn.functional as F


In [ ]:
def cosine_similarity(pseudo_model, client, target_data, dataset):
    pseudo_model.eval()
    client.eval()
    with torch.no_grad():
        x_a, _ = split_data(target_data, dataset)
        pseudo_output = pseudo_model(x_a)
        client_output = client(x_a)
    # 将特征展平
    pseudo_output = pseudo_output.view(pseudo_output.size(0), -1)
    client_output = client_output.view(client_output.size(0), -1)
    similarity = F.cosine_similarity(pseudo_output, client_output, dim=1)
    return similarity.mean().item()

In [29]:
target_bottom1,target_top = cifar_mobilenet(level=2)
bottom, _ = vgg16(level=2, batch_norm=True)

In [30]:
test_data = torch.ones(4, 3, 32, 32)
cosine_similarity = cosine_similarity(target_bottom1, bottom, test_data, 'cifar10')

In [31]:
output = target_bottom1(test_data)
print(output.shape)

torch.Size([4, 64, 16, 16])


In [32]:
print(cosine_similarity)

0.3270553946495056
